# Import WOS & Scopus to Infoscience

In [1]:
from data_pipeline.main import main
from datetime import datetime
import pandas as pd
import os
from data_pipeline.loader import Loader

In [ ]:
### Workflow metadata #####################
current_datetime = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
folder_path = "harvested-data"
path = os.path.join(folder_path, current_datetime)
if not os.path.exists(path):
    os.mkdir(path)
### Workflow execution #####################
start = "2024-09-07"
end = "2024-09-07"
df_metadata, df_authors, df_epfl_authors, df_unloaded = main(start_date=start, end_date=end)
### Save data #####################
df_metadata.to_csv(os.path.join(path, "ResearchOutput.csv"), index=False, encoding="utf-8")
df_authors.to_csv(os.path.join(path, "AddressesAndNames.csv"), index=False, encoding="utf-8")
df_epfl_authors.to_csv(os.path.join(path, "EpflAuthors.csv"), index=False, encoding="utf-8")
df_unloaded.to_csv(os.path.join(path, "UnloadedDuplicatedPublications.csv"), index=False, encoding="utf-8")

In [ ]:
### WIP Add step to create non-exitsing EPFL person's authorities in DSpace #####################

In [ ]:
### Upload data in DSpace #####################
# Loader
loader_instance = Loader(df_metadata, df_epfl_authors)
loaded_items = loader_instance.create_complete_publication()

In [ ]:
### WIP Add step to zip generated reports and send them by email #####################
loaded_items

***